# Read contents of .nwb file

In [ ]:
from pynwb import NWBHDF5IO

# Replace with the path to your .nwb file
nwb_file_path = 'data1.nwb'


# Open the .nwb file using PyNWB
with NWBHDF5IO(nwb_file_path, 'r') as io:
    nwbfile = io.read()

    # Access the acquisition group
    acquisition = nwbfile.acquisition

    # Print the names of all groups within the acquisition group
    print("Acquisition Groups:")
    for name, timeseries in acquisition.items():
        print(f" - {name}: {timeseries}")
    
    # Print all groups in stimulus (if any)
    if hasattr(nwbfile, 'stimulus'):
        print("\nStimulus Groups:")
        for name, sgroup in nwbfile.stimulus.items():
            print(f" - {name}: {sgroup}")

# Extract nwb file contents

In [ ]:
import numpy as np
from pynwb import NWBHDF5IO

# Function to extract data from a TimeSeries object
def extract_data(time_series):
    return {
        'data': np.array(time_series.data[:]),
        'timestamps': np.array(time_series.timestamps[:]) if time_series.timestamps else None,
        'unit': time_series.unit,
        'comments': time_series.comments
    }

# Path to your NWB file
nwb_file_path = 'data1.nwb'

# Open the .nwb file using PyNWB
with NWBHDF5IO(nwb_file_path, 'r') as io:
    nwbfile = io.read()

    # Extract acquisition groups
    acquisition_data = {}
    for name, timeseries in nwbfile.acquisition.items():
        acquisition_data[name] = extract_data(timeseries)

    # Extract stimulus groups if present
    stimulus_data = {}
    if hasattr(nwbfile, 'stimulus'):
        for name, timeseries in nwbfile.stimulus.items():
            stimulus_data[name] = extract_data(timeseries)

# Save extracted contents of nwb file as numpy

In [ ]:
for name, data in acquisition_data.items():
    np.save(f'{name}_acquisition.npy', data['data'])

for name, data in stimulus_data.items():
    np.save(f'{name}_stimulus.npy', data['data'])

# Save extracted contents of nwb file as dataframe csv

In [ ]:
import pandas as pd

for name, data in acquisition_data.items():
    # Check if data is two-dimensional
    if data['data'].ndim == 2:
        # Create column names for each dimension
        column_names = [f'{name}_dim_{i}' for i in range(data['data'].shape[1])]
    else:
        # For one-dimensional data, use a single column
        column_names = [f'{name}_value']

    df = pd.DataFrame(data['data'], columns=column_names)
    if data['timestamps'] is not None:
        df['timestamps'] = data['timestamps']
    df.to_csv(f'{name}_acquisition.csv', index=False)

for name, data in stimulus_data.items():
    # Check if data is two-dimensional
    if data['data'].ndim == 2:
        # Create column names for each dimension
        column_names = [f'{name}_dim_{i}' for i in range(data['data'].shape[1])]
    else:
        # For one-dimensional data, use a single column
        column_names = [f'{name}_value']

    df = pd.DataFrame(data['data'], columns=column_names)
    if data['timestamps'] is not None:
        df['timestamps'] = data['timestamps']
    df.to_csv(f'{name}_stimulus.csv', index=False)